In [2]:
import os
import vcgencmd as vc
import RPi.GPIO as gpio
from bottle   import route, run, template
from datetime import datetime

In [3]:
# see https://pypi.org/project/vcgencmd/ for usage

In [4]:
vccmd = vc.Vcgencmd()

In [5]:
vccmd.version()

'Dec  1 2021 15:02:46 \nCopyright (c) 2012 Broadcom\nversion 71bd3109023a0c8575585ba87cbb374d2eeb038f (clean) (release) (start_x)\n'

In [6]:
def get_temp():
    temp_C = vccmd.measure_temp()
    temp_F = temp_C * 9./5. +32.0
    return "{:0.2f} deg C, {:0.2f} deg F".format(temp_C, temp_F)
def get_time():
    return "{:%Y-%m-%d %H:%M:%S}".format(datetime.now())
def get_load_average():
    one, five, fifteen = os.getloadavg()
    return "Process Averages: {} (1m), {} (5m), {} (15m)".\
            format(one, five, fifteen)
def get_uptime():
    dd = os.popen('/usr/bin/uptime -p')
    return "Uptime: {}".format(dd.read()[3:-1])
def get_freq(obj='arm'):
    # obj = arm, core
    return "{} freq: {} GHz".\
        format(obj, vccmd.measure_clock(obj)/1.e9)
def get_processes(num=5):
    # see unix.stackexchange.com #13968 : sorting on cpu%
    # top result is the header, so get n + 1
    cmd = '/bin/ps aux --sort=-pcpu | head -n {}'.format(num + 1)
    dd  = os.popen(cmd)
    txt = dd.read()
    out = [k.split() for k in txt.split('\n')]
    fields = [0, 1, 2, 3, 8, 9, 10] # limit the output fields
    txt = ''
    for k in fields:
        txt += out[0][k] + '\t'
    for j in range(1, len(out) - 1):
        txt += '\n'
        for k in fields: # split affects only the last field
            txt += out[j][k].split('/')[-1] + '\t'
    return txt

In [7]:
print(get_temp(), get_time())

41.80 deg C, 107.24 deg F 2021-12-22 17:40:52


In [8]:
print(get_load_average())

Process Averages: 0.87 (1m), 0.58 (5m), 0.63 (15m)


In [9]:
print(get_uptime())

Uptime: 1 day, 44 minutes


In [10]:
print(get_freq('arm'), get_freq('core'))

arm freq: 1.500398464 GHz core freq: 0.500000992 GHz


In [27]:
print(get_processes(10))

USER	PID	%CPU	%MEM	START	TIME	COMMAND	
pi	27990	23.2	2.6	13:31	58:07	chromium-browser-v7	
pi	18066	5.4	4.1	12:50	15:55	firefox-esr	
pi	27952	4.7	2.3	13:31	11:48	chromium-browser-v7	
pi	23048	4.0	0.0	Dec21	56:26	htop	
pi	16687	2.9	0.2	12:44	8:36	lxtask	
pi	27888	2.7	2.0	13:31	6:48	chromium-browser-v7	
root	681	1.9	1.2	Dec21	28:18	Xorg	
pi	11669	0.8	4.4	Dec21	12:45	firefox-esr	
pi	27804	0.8	0.7	13:31	2:02	python3	
pi	2547	0.5	0.4	Dec21	8:01	lxterminal	


In [12]:
vccmd.get_sources?

Signature: vccmd.get_sources(typ)
Docstring: <no docstring>
File:      /usr/local/lib/python3.7/dist-packages/vcgencmd/vcgencmd.py
Type:      method


In [13]:
#get_sources("mem") will return ["arm", "gpu"] which are the sources/arguments for 
#get_mem(source) that returns the memory of the source passed as argument to it.
# the below dict from /usr/local/lib/python3.7/dist-packages/vcgencmd/vcgencmd.py
sources = {
  "clock": ["arm", "core","isp", "v3d", "uart", "pwm", "emmc", "pixel", 
            "vec", "hdmi", "dpi"], # deleted H264: gives error
  "volts": ["core", "sdram_c", "sdram_i", "sdram_p"],
  "mem":   ["arm", "gpu"],
  "codec": ["agif", "flac", "h263", "h264", "mjpa", "mjpb", "mjpg", "mpg2", 
            "mpg4", "mvc0", "pcm", "thra", "vorb", "vp6", "vp8", "wmv9", "wvc1"],
  "display_id": [0, 1, 2, 3, 7]
  }

In [14]:
dd = vccmd.get_sources('mem')
for k in dd:
    print(k, vccmd.get_mem(k))

arm 896
gpu 128


In [15]:
dd = vccmd.get_sources('clock')
for k in dd:
    # H264 produces an error
    if (k != 'H264'): print(k, vccmd.measure_clock(k))

arm 1500345728
core 500000992
isp 0
v3d 500000992
uart 48001464
pwm 107143064
emmc 250000496
pixel 81000000
vec 0
hdmi 0
dpi 0


In [16]:
dd = vccmd.get_sources('volts')
for k in dd:
    print(k, vccmd.measure_volts(k))

core 0.86
sdram_c 1.1
sdram_i 1.1
sdram_p 1.1


In [17]:
dd = vccmd.get_sources('codec')
for k in dd:
    print(k, vccmd.codec_enabled(k))

agif False
flac False
h263 False
h264 False
mjpa False
mjpb False
mjpg False
mpg2 False
mpg4 False
mvc0 False
pcm False
thra False
vorb False
vp6 False
vp8 False
wmv9 False
wvc1 False


In [18]:
dd = vccmd.get_sources('display_id')
for k in dd:
    print(k, vccmd.display_power_state(k))

0 off
1 off
2 on
3 off
7 on


In [19]:
vccmd.get_lcd_info()

{'height': '1920', 'width': '1080', 'depth': '24\n'}

In [20]:
vccmd.dispmanx_list()

{'display': '7',
 'format': 'ARGB8888',
 'transform': '0',
 'layer': '1',
 'resolution': '64x64',
 'src': '0,0,64,64',
 'dst': '439,421,64,64',
 'cost': '125',
 'lbm': '0'}

In [28]:
#vccmd.otp_dump()

In [22]:
#dev = os.popen('bin/ps -a', 'r', 1)
dev = os.popen('/opt/vc/bin/vcgencmd measure_temp')  # this works
dev.read()
#dev.close()

"temp=45.2'C\n"

In [23]:
os.getlogin()

'pi'

In [24]:
os.listdir()

['basic_testing.ipynb',
 'README.md',
 '.git',
 'LICENSE',
 '.gitignore',
 '.ipynb_checkpoints']

In [ ]:
#os.sysconf_names # prints out a lot

# now play with bottle

In [30]:
def index(name='RPi Status'):
    dt = get_time()
    tc = get_temp()
    return template('<b>TIME: {{t}},  CPU TEMP:  {{tc}}</b>',
                    t=dt, tc=tc)

In [31]:
tt = index()

In [32]:
tt

'<b>TIME: 2021-12-22 17:50:43,  CPU TEMP:  40.90 deg C, 105.62 deg F</b>'